# **csv Data Load**

- 개인 parsed dataset csv 파일 경로에 맞추어 불러와주세요
- df 변수 저장 후 아래 각 케이스마다 코드 셀을 그대로 복붙하면 해당 데이터셋을 사용할 수 있습니다

In [ ]:
# 개인 디렉토리에 맞추어 경로를 설정
from google.colab import drive
drive.mount('/content/gdrive/')

import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/인공지능 커리어패스/parsed_enron_all.csv')

Mounted at /content/gdrive/


# **[1] 전체 Enron 데이터셋**
- 변수명: enron_dataset
- To 항목 공란(NaN) 삭제
- 자기자신에게 보낸 메일 삭제
- Date 정보 utc 기준으로 통일

In [ ]:
def clean_enron_emails(df):
    df_processed = df.copy()
    initial_count = len(df_processed)
    print(f"초기 데이터 개수: {initial_count}")
    print("-" * 50)

    # 1. 'To' 항목이 공란(NaN)인 메일 개수 계산 (삭제 대상)
    to_nan_mask = df_processed['To'].isna()
    to_nan_count = to_nan_mask.sum()
    print(f"To 항목이 공란(NaN)인 메일")
    print(f"삭제 대상 메일 개수: {to_nan_count}")
    print("-" * 50)

    # 2 & 3. process_recipients 함수를 적용하여 'To' 필드 처리
    def process_recipients(row):
        from_email = str(row['From']).strip().lower()
        to_field = row['To']

        # To 필드가 문자열이 아닌 경우(NaN 등)는 변경하지 않음
        if not isinstance(to_field, str) or not to_field:
            return to_field

        recipients = {email.strip().lower() for email in to_field.split(',')}

        if from_email in recipients:
            if len(recipients) == 1:
                return None  # 자기 자신에게만 보낸 경우 -> 삭제 대상
            else:
                recipients.remove(from_email)
                return ', '.join(recipients) # 자기 자신과 타인에게 보낸 경우 -> 수정
        else:
            return to_field # 그 외 -> 유지
    df_processed['To_processed'] = df_processed.apply(process_recipients, axis=1)

    # 2. 자기 자신에게만 보낸 메일 개수 계산 (삭제 대상)
    # To_processed가 None이면서, 원래 To는 NaN이 아니었던 경우
    self_only_mask = df_processed['To_processed'].isna() & df_processed['To'].notna()
    self_only_count = self_only_mask.sum()
    print(f"자기 자신에게만 보낸 메일")
    print(f"삭제 대상 메일 개수: {self_only_count}")
    print("-" * 50)

    # 3. 자기 자신과 다른 사람에게 보낸 메일 개수 계산 (수정 대상)
    modified_mask = (df_processed['To_processed'].notna()) & (df_processed['To_processed'] != df_processed['To'])
    modified_count = modified_mask.sum()
    print(f"자기 자신과 동시에 타인에게 보낸 메일의 수신자 목록에서 자기 자신을 제외")
    print(f"수정된 메일 개수: {modified_count}")
    print("-" * 50)

    # 4. 최종 데이터셋 생성
    # 삭제 대상(To가 NaN이거나, 자기 자신에게만 보낸 메일)을 모두 제외
    final_df = df_processed[~to_nan_mask & ~self_only_mask].copy()

    # 처리된 'To_processed' 컬럼을 'To' 컬럼에 반영
    final_df['To'] = final_df['To_processed']
    # 원본 컬럼 목록을 유지하기 위해 불필요한 임시 컬럼 삭제
    final_df = final_df[df.columns]

    final_count = len(final_df)
    total_removed = initial_count - final_count

    print(f"최종 데이터 개수: {final_count}")
    print(f"총 {total_removed}개의 메일이 삭제되었습니다.")

    return final_df


enron_dataset = clean_enron_emails(df)

# Date 정보를 utc 기준으로 통일
enron_dataset['Date'] = pd.to_datetime(
    enron_dataset['Date'],
    format='%a, %d %b %Y %H:%M:%S %z',
    errors='coerce',
    utc=True
)

초기 데이터 개수: 517396
--------------------------------------------------
To 항목이 공란(NaN)인 메일
삭제 대상 메일 개수: 21847
--------------------------------------------------
자기 자신에게만 보낸 메일
삭제 대상 메일 개수: 16295
--------------------------------------------------
자기 자신과 동시에 타인에게 보낸 메일의 수신자 목록에서 자기 자신을 제외
수정된 메일 개수: 12160
--------------------------------------------------
최종 데이터 개수: 479254
총 38142개의 메일이 삭제되었습니다.


In [ ]:
# [1] enron_dataset의 구성을 확인
enron_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 479254 entries, 0 to 517395
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   Message-ID  479254 non-null  object             
 1   Date        479254 non-null  datetime64[ns, UTC]
 2   From        479254 non-null  object             
 3   To          479254 non-null  object             
 4   Subject     460708 non-null  object             
 5   X-From      479247 non-null  object             
 6   X-To        479247 non-null  object             
 7   X-cc        117535 non-null  object             
 8   X-bcc       168 non-null     object             
 9   X-Folder    479247 non-null  object             
 10  X-Origin    479247 non-null  object             
 11  X-FileName  476689 non-null  object             
 12  Body        479254 non-null  object             
dtypes: datetime64[ns, UTC](1), object(12)
memory usage: 51.2+ MB


# **[2] 특정 메일 주소의 보낸메일함**
- 변수명: personal_sent_dataset
- 필요한 사전 정의 데이터셋: [1] enron_dataset
- 필요 파라미터: [1] enron_dataset 과 특정메일 주소(str)

In [ ]:
def create_outbox(df, email_address):
    # 이메일 주소 정규화 (공백 제거, 소문자 변환)
    email_address = email_address.strip().lower()

    # 'From' 컬럼이 해당 이메일 주소와 일치하는 행을 필터링
    outbox_df = df[df['From'].str.strip().str.lower() == email_address].copy()

    ###################################################################
    ## 실제 사용하실 칼럼만을 설정하여 데이터셋을 구축하시면 됩니다. ##
    ## 임의로 Message-ID, From, To, Date, Body 칼럼만 담아냈습니다.  ##
    ###################################################################
    outbox_df = outbox_df[['Message-ID', 'From', 'To', 'Date', 'Body']]

    return outbox_df

##############################################################
## Guide: 특정메일 주소(str)를 파라미터로 넣어주시면 됩니다 ##
##############################################################
personal_sent_dataset = create_outbox(enron_dataset, "jeff.dasovich@enron.com")

In [ ]:
# From 항목에 특정메일 주소만 담겨있는 것을 확인할 수 있습니다
personal_sent_dataset.head()

,Message-ID,From,To,Date,Body
8657,<31563977.1075863589691.JavaMail.evans@thyme>,jeff.dasovich@enron.com,"sarah.novosel@enron.com, james.steffes@enron.com",2000-08-16 18:22:00+00:00,Tim/Bob:\n\nAttached is the letter that we sen...
8677,<4985901.1075863590149.JavaMail.evans@thyme>,jeff.dasovich@enron.com,"dennis.benevides@enron.com, roger.yang@enron.c...",2000-08-11 12:27:00+00:00,---------------------- Forwarded by Jeff Dasov...
8688,<21881287.1075863590378.JavaMail.evans@thyme>,jeff.dasovich@enron.com,"richard.shapiro@enron.com, sandra.mccubbin@enr...",2000-08-10 10:46:00+00:00,Deregulation: `Conspiracy of Incompetence'\nEv...
8689,<2219999.1075863590405.JavaMail.evans@thyme>,jeff.dasovich@enron.com,"susan.mara@enron.com, mona.petrochko@enron.com...",2000-08-10 10:12:00+00:00,There appears to be a pattern forming. The ad...
8696,<21947511.1075863590609.JavaMail.evans@thyme>,jeff.dasovich@enron.com,"james.steffes@enron.com, richard.shapiro@enron...",2000-08-09 10:54:00+00:00,THE WALL STREET JOURNAL / CALIFORNIA \n\nCriti...


# **[3] 특정 메일주소의 받은메일함**
- 변수명: personal_received_dataset
- 필요한 사전 정의 데이터셋: [1] enron_dataset
- 필요 파라미터: [1] enron_dataset 과 특정메일 주소(str)

In [ ]:
def create_inbox(df, email_address):
    # 이메일 주소 정규화 (공백 제거, 소문자 변환)
    email_address = email_address.strip().lower()

    # 'To' 컬럼에 해당 이메일 주소를 포함하는 행을 필터링 (NaN 값은 제외)
    inbox_df = df[df['To'].str.lower().str.contains(email_address, na=False)].copy()

    ###################################################################
    ## 실제 사용하실 칼럼만을 설정하여 데이터셋을 구축하시면 됩니다. ##
    ## 임의로 Message-ID, From, To, Date, Body 칼럼만 담아냈습니다.  ##
    ###################################################################
    inbox_df = inbox_df[['Message-ID', 'From', 'To', 'Date', 'Body']]

    return inbox_df

##############################################################
## Guide: 특정메일 주소(str)를 파라미터로 넣어주시면 됩니다 ##
##############################################################
personal_received_dataset = create_inbox(enron_dataset, "jeff.dasovich@enron.com")

In [ ]:
# To 항목에 해당 메일주소와 동시에 함께 수신한 다른 메일주소도 담겨있을 수 있습니다
personal_received_dataset.head()

,Message-ID,From,To,Date,Body
418,<21381996.1075855686304.JavaMail.evans@thyme>,phillip.allen@enron.com,"christi.nicolay@enron.com, james.steffes@enron...",2000-12-13 15:04:00+00:00,Attached are two files that illustrate the fo...
723,<7400905.1075855665946.JavaMail.evans@thyme>,phillip.allen@enron.com,"christi.nicolay@enron.com, james.steffes@enron...",2000-12-13 15:04:00+00:00,Attached are two files that illustrate the fo...
777,<14286989.1075855666059.JavaMail.evans@thyme>,stephanie.miller@enron.com,jeff.dasovich@enron.com,2000-12-13 13:26:00+00:00,Any merit to mentioning that there has been an...
810,<31689466.1075855666131.JavaMail.evans@thyme>,sarah.novosel@enron.com,"christi.nicolay@enron.com, james.steffes@enron...",2000-12-13 10:28:00+00:00,"Everyone: \n\nI forgot to mention, these are ..."
821,<28916908.1075855666155.JavaMail.evans@thyme>,sarah.novosel@enron.com,"james.steffes@enron.com, joe.hartsoe@enron.com...",2000-12-13 10:17:00+00:00,Please review the attached draft Enron comment...


# **[4] 특정 메일주소의 통합(보낸+받은)메일함**
- 변수명: personal_combined_dataset
- 필요한 사전 정의 데이터셋: [1] enron_dataset, [2] personal_sent_dataset, [3] personal_received_dataset
- 필요 파라미터: [1] enron_dataset 과 특정메일 주소(str)
- 메일ID 기준으로 혹시 모를 중복된 메일까지 제거(확인결과, 실제 제거되는 메일 수는 극소수)
- date 기준으로 데이터셋 재정렬

In [ ]:
def create_combined_box(df, email_address):
    # 위에서 정의한 함수들을 재사용하여 발신 및 수신 메일함 생성
    outbox_df = create_outbox(df, email_address)
    inbox_df = create_inbox(df, email_address)

    # 두 데이터프레임을 하나로 합침
    combined_df = pd.concat([outbox_df, inbox_df])

    # Message-ID 컬럼을 기준으로 실제 중복 메일만 제거
    combined_df_deduped = combined_df.drop_duplicates(subset=['Message-ID']).reset_index(drop=True)

    return combined_df_deduped

##############################################################
## Guide: 특정메일 주소(str)를 파라미터로 넣어주시면 됩니다 ##
##############################################################
personal_combined_dataset = create_combined_box(enron_dataset, "jeff.dasovich@enron.com")
# date 기준으로 데이터셋 재정렬
personal_combined_dataset = personal_combined_dataset.sort_values(by='Date').reset_index(drop=True)

In [ ]:
# date 기준으로 정렬된 통합메일함(보낸메일함+받은메일함)
personal_combined_dataset.head(10)

,Message-ID,From,To,Date,Body
0,<24525978.1075842950545.JavaMail.evans@thyme>,jeff.dasovich@enron.com,gramlr@pjm.com,1980-01-01 00:00:00+00:00,Agreed. I'm on vacation this week (in the high...
1,<25476052.1075842950567.JavaMail.evans@thyme>,jeff.dasovich@enron.com,ginger.dernehl@enron.com,1980-01-01 00:00:00+00:00,Hi. I'm sorry for not responding. I'm actual...
2,<16094844.1075842948326.JavaMail.evans@thyme>,jeff.dasovich@enron.com,"janel.guerrero@enron.com, kirk@cea.sfex.com",1980-01-01 00:00:00+00:00,"Yes on regulatory. Discussion, not power poin..."
3,<30631880.1075843060123.JavaMail.evans@thyme>,jeff.dasovich@enron.com,paul.kaufman@enron.com,1980-01-01 00:00:00+00:00,Sounds good.\n\n\n\n\tPaul Kaufman@ECT\n\t12/0...
4,<11972760.1075842947758.JavaMail.evans@thyme>,jeff.dasovich@enron.com,susan.scott@enron.com,1980-01-01 00:00:00+00:00,"Hey, you be talkin' to gas trash, counsel.\n\n..."
5,<7811304.1075842930531.JavaMail.evans@thyme>,bruno.gaillard@enron.com,"susan.mara@enron.com, mona.petrochko@enron.com...",1998-09-01 18:20:00+00:00,Ignor previous message again
6,<3047299.1075842930637.JavaMail.evans@thyme>,martin.esarte@clorox.com,jeff.dasovich@enron.com,1999-09-20 20:21:00+00:00,I could also do a six o'clock conference call....
7,<23125303.1075843087683.JavaMail.evans@thyme>,martin.esarte@clorox.com,jeff.dasovich@enron.com,1999-09-20 20:21:00+00:00,I could also do a six o'clock conference call....
8,<20720002.1075842930744.JavaMail.evans@thyme>,orders@amazon.com,jeff.dasovich@enron.com,1999-09-21 23:50:00+00:00,Thank you for ordering from Amazon.com!\n\nYou...
9,<23006419.1075843183310.JavaMail.evans@thyme>,orders@amazon.com,jeff.dasovich@enron.com,1999-09-21 23:50:00+00:00,Thank you for ordering from Amazon.com!\n\nYou...


# **[5] 최근 n일(입력) 기간 동안의 메일함(Train Dataset, n=90)**
> 기본값은 Jeff의 2002-01-01 기준 앞의 90일 동안의 통합메일함
- 변수명: recent_ndays_dataset
- 필요한 사전 정의 데이터셋: [4] personal_combined_dataset
- 필요 파라미터: [4] personal_combined_dataset, 현재날짜(str), 최근n일에 대한 n값(int)
- 인자로 입력한 현재날짜 기준으로 최근 n일 동안의 메일함을 구축

In [ ]:
def get_recent_emails(df, current_date, n):
    # 1. 입력받은 날짜 문자열을 datetime 객체로 변환
    end_date = pd.to_datetime(current_date)

    # 2. n일 전의 시작 날짜를 계산
    start_date = end_date - pd.Timedelta(days=n)

    # 3. 데이터셋의 'Date' 컬럼(UTC)과 비교하기 위해 기준 날짜들도 UTC로 통일
    start_date_utc = start_date.tz_localize('UTC')
    end_date_utc = end_date.tz_localize('UTC')

    # 4. 시작 날짜와 종료 날짜 사이의 데이터를 필터링
    # [start_date <= email_date < end_date] 범위의 메일을 선택
    mask = (df['Date'] >= start_date_utc) & (df['Date'] < end_date_utc)
    recent_ndays_dataset = df[mask].copy()

    return recent_ndays_dataset

##############################################################
## Guide: 특정메일 주소(str)를 파라미터로 넣어주시면 됩니다 ##
##############################################################
personal_combined_dataset = create_combined_box(enron_dataset, "jeff.dasovich@enron.com")
# date 기준으로 데이터셋 재정렬
personal_combined_dataset = personal_combined_dataset.sort_values(by='Date').reset_index(drop=True)

######################################################################
## Guide: 특정메일주소로 personal_combined_dataset을 먼저 구축하고, ##
##        현재시각을 'YYYY-MM-DD' 형태로 파라미터 입력              ##
##        최근 n일에 해당하는 n값을 정수 형태로 파라미터 입력       ##
######################################################################
# Jeff Train Dataset 기준 2001-01-01 기준 앞의 90일을 다루기로 7/14 회의에서 결정
date = '2002-01-01'
n = 90
recent_ndays_dataset = get_recent_emails(personal_combined_dataset, date, n)
print(f"추출된 최근 {n}일간의 메일 수: {len(recent_ndays_dataset)}")

추출된 최근 90일간의 메일 수: 2260


In [ ]:
# 최근 n일 범위 중 가장 오래된 메일 쪽 데이터
recent_ndays_dataset.head()

,Message-ID,From,To,Date,Body
23319,<10518649.1075851652101.JavaMail.evans@thyme>,ttt@cpuc.ca.gov,jeff.dasovich@enron.com,2001-10-03 00:13:53+00:00,Jeff - a business question:\n\nAn ORA staffer ...
23320,<30407895.1075852563048.JavaMail.evans@thyme>,scottwl@hotmail.com,jeff.dasovich@enron.com,2001-10-03 03:19:10+00:00,I think that looks a little dubious to the res...
23321,<23490285.1075852595389.JavaMail.evans@thyme>,jeff.dasovich@enron.com,"thompson@enron.com, ttt@cpuc.ca.gov",2001-10-03 04:46:15+00:00,"come on, you're kidding, right?\n\n-----Origin..."
23322,<6795632.1075851652176.JavaMail.evans@thyme>,robert.frank@enron.com,jeff.dasovich@enron.com,2001-10-03 10:50:24+00:00,Hey Jeff -see memo attached; it contains good ...
23323,<24214413.1075852595366.JavaMail.evans@thyme>,jeff.dasovich@enron.com,"lisa.mellencamp@enron.com, michael.tribolet@en...",2001-10-03 13:50:23+00:00,I'll be in the office very shortly. My conver...


In [ ]:
# 최근 n일 범위 중 가장 최신 메일 쪽 데이터
recent_ndays_dataset.tail()

,Message-ID,From,To,Date,Body
25574,<15759619.1075859214587.JavaMail.evans@thyme>,ginger.dernehl@enron.com,"paul.kaufman@enron.com, jeff.dasovich@enron.co...",2001-12-28 21:01:42+00:00,I wanted to wish each of you a Happy New Year ...
25575,<24652099.1075859194938.JavaMail.evans@thyme>,schwabalerts.marketupdates@schwab.com,jeff.dasovich@enron.com,2001-12-28 22:59:26+00:00,"Charles Schwab & Co., Inc.\nEmail Alert\n\nInt..."
25576,<28336342.1075859214635.JavaMail.evans@thyme>,douglass@energyattorney.com,"johnson.tamara@enron.com, jeff.dasovich@enron....",2001-12-29 08:12:48+00:00,Attached are summaries of 13 recent PG&E advic...
25577,<13415897.1075859195353.JavaMail.evans@thyme>,eldon@direcpc.com,"e-mail <'.'andre@enron.com>, e-mail <'.'andy@e...",2001-12-31 06:13:34+00:00,Results:\n1st Place - Prentice (Congratulation...
25578,<25550111.1075859194913.JavaMail.evans@thyme>,schwabalerts.marketupdates@schwab.com,jeff.dasovich@enron.com,2001-12-31 14:26:45+00:00,"Charles Schwab & Co., Inc.\nEmail Alert\n\nMor..."


# **[6] Jeff의 받은메일함(Test Dataset)**
- 변수명: future_ndays_dataset
- 필요한 사전 정의 데이터셋: [3] personal_received_dataset
- 필요 파라미터: [3] personal_received_dataset, 현재날짜(str), 이후n일에 대한 n값(int)
- 인자로 입력한 현재날짜 기준으로 n일 이후까지의 받은메일함을 구축

In [ ]:
def get_emails_for_ndays_after(df, start_date_str, n):
    # 1. 입력받은 시작 날짜 문자열을 datetime 객체로 변환
    start_date = pd.to_datetime(start_date_str)

    # 2. n일 후의 종료 날짜를 계산
    end_date = start_date + pd.Timedelta(days=n + 1)

    # 3. 데이터셋의 'Date' 컬럼(UTC)과 비교하기 위해 기준 날짜들도 UTC 기준으로 통일
    start_date_utc = start_date.tz_localize('UTC')
    end_date_utc = end_date.tz_localize('UTC')

    # 4. 시작 날짜와 종료 날짜 사이의 데이터를 필터링
    mask = (df['Date'] >= start_date_utc) & (df['Date'] < end_date_utc)
    future_ndays_dataset = df.loc[mask].copy()

    return future_ndays_dataset

##############################################################
## Guide: 특정메일 주소(str)를 파라미터로 넣어주시면 됩니다 ##
##############################################################
personal_received_dataset = create_inbox(enron_dataset, "jeff.dasovich@enron.com")
# date 기준으로 데이터셋 재정렬
personal_received_dataset = personal_received_dataset.sort_values(by='Date').reset_index(drop=True)

#######################################################################
## Guide: 특정메일주소로 personal_receivedd_dataset을 먼저 구축하고, ##
##        현재시각을 'YYYY-MM-DD' 형태로 파라미터 입력               ##
##        n일 이후에 해당하는 n값을 정수 형태로 파라미터 입력        ##
#######################################################################
# Jeff Train Dataset 기준 2002-01-01 기준 앞의 90일을 다루기로 7/14 회의에서 결정
date = '2002-01-01'
n = 20
future_ndays_dataset = get_emails_for_ndays_after(personal_received_dataset, date, n)
print(f"추출된 {n}일 이후의 받은 메일 수: {len(future_ndays_dataset)}")

추출된 20일 이후의 받은 메일 수: 19


# **직접 라벨링 할 데이터셋**

In [ ]:
email_text_list = []

for index, row in future_ndays_dataset.iterrows():
    # 데이터셋에서 모든 내용을 텍스트로 전환
    email_block = (
        f"From: {row['From']}\n"
        f"To: {row['To']}\n"
        f"Date: {row['Date']}\n"
        f"Body: {row['Body']}"
    )
    email_text_list.append(email_block)

# 각 이메일에 대한 텍스트 구분 ("\n\n===separatrix===\n\n")
received_emails = "\n\n===separatrix===\n\n".join(email_text_list)
print(len(email_text_list))
print("===================")
print(received_emails)

19
From: schwabalerts.marketupdates@schwab.com
To: jeff.dasovich@enron.com
Date: 2002-01-02 14:25:26+00:00
Body: Charles Schwab & Co., Inc.
Email Alert

Morning Market View(TM) 
for Wednesday, January 2, 2002
as of 9:30AM EST
Information provided by Schwab Center for Investment Research


STOCKS SLIGHTLY HIGHER TO KICK OFF NEW YEAR

Against the backdrop of little corporate or economic news, 
stocks are poised to open slightly higher, with technology 
leading the way following an upbeat Semiconductor Industry 
Association report. Chipmaker Micron Technology (MU,31,f2) is 
higher in premarket activity following news that a potential 
partner raised DRAM prices by 30%. Treasuries are slightly 
lower, while global markets are mixed as today marks the 
official transition to the euro currency in the euro-region.

Chipmakers could get a boost this morning after the 
Semiconductor Industry Association (SIA) reported that global 
sales of semiconductors rose 1.6% in November to $10.6 billion, 